# Lab 2: Bronze Layer - Raw Streaming Data

## 🎯 **Learning Objectives:**
- Ingest stock trade data từ Kafka vào Iceberg Bronze table
- Setup Spark Structured Streaming với Kafka source
- Write streaming data to Iceberg tables
- Understand checkpointing và fault tolerance
- Verify raw data trong Bronze layer

## 📚 **Key Concepts:**
1. **Bronze Layer**: Raw, unprocessed data từ source
2. **Kafka Source**: Real-time event stream
3. **Iceberg Sink**: Write streaming data to Iceberg
4. **Checkpointing**: Fault tolerance cho streaming
5. **Schema**: Stock trade event structure


In [ ]:
# Install and Import Dependencies
%pip install pyspark findspark pandas numpy pyarrow kafka-python

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.streaming import StreamingQuery
import json
import time

print("✅ Dependencies installed and imported successfully!")


In [ ]:
# Initialize Spark Session for Streaming Lakehouse
spark = SparkSession.builder \
    .appName("StreamingLakehouseBronze") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/bronze_checkpoint") \
    .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "false") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Set log level
spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark Streaming Session initialized!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔗 Master URL: {spark.sparkContext.master}")

# Configuration
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"
KAFKA_TOPIC = "stock-trades"
BRONZE_TABLE_PATH = "/warehouse/bronze/trades"

print(f"\n📡 Configuration:")
print(f"   Kafka: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"   Topic: {KAFKA_TOPIC}")
print(f"   Bronze Path: {BRONZE_TABLE_PATH}")


## Exercise 1: Define Stock Trade Schema

### Stock Trade Event Schema
```json
{
  "trade_id": "TRD_001",
  "symbol": "AAPL",
  "price": 175.50,
  "volume": 100,
  "timestamp": "2025-01-15T10:30:00Z",
  "trade_type": "BUY",
  "exchange": "NASDAQ"
}
```


In [ ]:
# Define Stock Trade Schema
print("📋 Exercise 1: Define Stock Trade Schema")
print("=" * 60)

trade_schema = StructType([
    StructField("trade_id", StringType()),
    StructField("symbol", StringType()),
    StructField("price", DoubleType()),
    StructField("volume", IntegerType()),
    StructField("timestamp", StringType()),
    StructField("trade_type", StringType()),
    StructField("exchange", StringType())
])

print("✅ Stock Trade Schema defined:")
trade_schema.printTreeString()

print("\n💡 Note: Bronze layer stores raw data as-is")
print("   - No transformations")
print("   - No validations")
print("   - Preserves original format for replay capability")


## Exercise 2: Read from Kafka Stream

### Steps:
1. Read stream từ Kafka topic
2. Parse JSON messages
3. Extract và cast fields
4. Add Kafka metadata (partition, offset)


In [ ]:
# Read from Kafka Stream
print("📥 Exercise 2: Read from Kafka Stream")
print("=" * 60)

print(f"\n1️⃣ Reading from Kafka topic: {KAFKA_TOPIC}")

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", KAFKA_TOPIC) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

print("✅ Kafka stream created!")
print("\n📋 Kafka stream schema:")
kafka_stream.printSchema()

# Parse JSON from Kafka value
print("\n2️⃣ Parsing JSON messages:")

parsed_stream = kafka_stream \
    .select(
        col("key").cast("string").alias("kafka_key"),
        col("value").cast("string").alias("json_value"),
        col("timestamp").alias("kafka_timestamp"),
        col("partition"),
        col("offset")
    ) \
    .select(
        col("kafka_key"),
        from_json(col("json_value"), trade_schema).alias("data"),
        col("kafka_timestamp"),
        col("partition"),
        col("offset")
    ) \
    .select(
        col("kafka_key"),
        col("data.*"),
        col("kafka_timestamp").alias("kafka_ingestion_time"),
        col("partition").alias("kafka_partition"),
        col("offset").alias("kafka_offset")
    ) \
    .withColumn(
        "event_timestamp",
        to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss'Z'")
    )

print("✅ Parsed stream schema:")
parsed_stream.printSchema()

print("\n💡 Bronze layer includes:")
print("   - Original trade data (from Kafka)")
print("   - Kafka metadata (partition, offset, ingestion time)")
print("   - Event timestamp (from data)")


## Exercise 3: Write to Iceberg Bronze Table

### Key Points:
- **Format**: Iceberg (unified storage)
- **Checkpointing**: For fault tolerance
- **Trigger**: Processing time (e.g., 10 seconds)
- **Output Mode**: Append (new data only)

### Note:
Full Iceberg functionality requires Iceberg Spark runtime JAR.
For demo, we'll show the pattern. In production, configure:
```python
.config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
.config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
```


In [ ]:
# Write to Iceberg Bronze Table
print("💾 Exercise 3: Write to Iceberg Bronze Table")
print("=" * 60)

print(f"\n1️⃣ Writing to Bronze layer: {BRONZE_TABLE_PATH}")

# For demo: Write to Parquet (Iceberg pattern)
# In production with Iceberg JAR, use:
# .format("iceberg")
# .option("path", BRONZE_TABLE_PATH)
# .option("checkpointLocation", "/tmp/bronze_checkpoint")

bronze_query = parsed_stream \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", BRONZE_TABLE_PATH) \
    .option("checkpointLocation", "/tmp/bronze_checkpoint") \
    .trigger(processingTime='10 seconds') \
    .start()

print("✅ Bronze streaming query started!")
print(f"   Writing to: {BRONZE_TABLE_PATH}")
print(f"   Checkpoint: /tmp/bronze_checkpoint")
print(f"   Trigger: Every 10 seconds")

print("\n💡 With Iceberg JAR, use:")
print("   .format('iceberg')")
print("   .option('path', 'warehouse.bronze.trades')")
print("   .option('checkpointLocation', '/tmp/bronze_checkpoint')")

print("\n⚠️  Query is running. To stop: bronze_query.stop()")
print("   To check status: bronze_query.status")


## Exercise 4: Verify Bronze Data

### Check:
- Data is being written
- Schema is correct
- Kafka metadata is preserved
- Event timestamps are correct


In [ ]:
# Verify Bronze Data
print("🔍 Exercise 4: Verify Bronze Data")
print("=" * 60)

# Wait a bit for data to arrive
print("\n⏳ Waiting for data to be written...")
time.sleep(15)

# Read Bronze data (batch read)
print("\n1️⃣ Reading Bronze data:")

try:
    bronze_df = spark.read.parquet(BRONZE_TABLE_PATH)
    
    print(f"✅ Bronze data found!")
    print(f"   Total records: {bronze_df.count()}")
    
    print("\n2️⃣ Sample Bronze data:")
    bronze_df.show(10, truncate=False)
    
    print("\n3️⃣ Schema:")
    bronze_df.printSchema()
    
    print("\n4️⃣ Data summary:")
    bronze_df.select("symbol", "price", "volume", "trade_type").summary().show()
    
    print("\n5️⃣ Records by symbol:")
    bronze_df.groupBy("symbol").count().orderBy(desc("count")).show()
    
except Exception as e:
    print(f"⚠️  Error reading Bronze data: {e}")
    print("   Make sure:")
    print("   1. Kafka producer is running (stock_trade_simulator.py)")
    print("   2. Streaming query has processed some batches")
    print("   3. Wait a bit longer for data to arrive")


## Summary

### ✅ What we learned:
1. **Bronze Layer**: Raw, unprocessed data từ Kafka
2. **Kafka Integration**: Read streaming data từ Kafka topics
3. **Schema Definition**: Define structure cho stock trade events
4. **Iceberg Write**: Write streaming data to Iceberg (pattern)
5. **Checkpointing**: Fault tolerance cho streaming queries
6. **Verification**: Check data trong Bronze layer

### 🎯 Key Takeaways:
- **Bronze = Raw**: No transformations, preserves original data
- **Replay Capability**: Có thể reprocess từ Bronze nếu cần
- **Kafka Metadata**: Preserve partition, offset, ingestion time
- **Unified Storage**: Iceberg cho phép query real-time và historical

### 🚀 Next Steps:
- Lab 3: Silver Layer (cleaning, deduplication)
- Lab 4: Gold Layer (aggregations)
- Lab 5: Unified batch + streaming
